In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from copy import deepcopy
from sklearn.metrics import silhouette_score
import pandas as pd 
import torch.optim as optim
from braingeneers.analysis import SpikeData
import os
import json
import pytorch_lightning as pl

In [2]:
waveform_files = ["data/" + f for f in os.listdir("./data") if f.endswith('.npy')]
meta_files = [f.replace('.npy', '.json') for f in waveform_files]

waveforms = []
isi_dist = []

for wf, mf in zip(waveform_files, meta_files):
    waveforms.append(np.load(wf))
    
    with open(mf, 'r') as file:
        json_data = json.load(file)

    sd = SpikeData([json_data[key]['train'] for key in json_data.keys()])
    trains = sd.train

    all_isi = sd.interspike_intervals()
    
    for isi in all_isi:
        hist, edges = np.histogram(isi, bins=50, density=True)
        isi_dist.append(hist)


In [3]:
waveforms = np.concatenate(waveforms, axis=0)

In [4]:
isi_dist[0]

array([0.65862158, 0.09717368, 0.06478245, 0.0863766 , 0.0431883 ,
       0.04858684, 0.03239123, 0.0431883 , 0.03778976, 0.01619561,
       0.00539854, 0.01619561, 0.01619561, 0.01079708, 0.        ,
       0.00539854, 0.01619561, 0.01619561, 0.        , 0.        ,
       0.01079708, 0.        , 0.00539854, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00539854])

In [5]:
class EphysDataset(Dataset):
    def __init__(self, waveforms, isi_dists, normalize=True):
        self.waveforms = np.array(waveforms)
        self.isi_dists = np.array(isi_dists)
        
        assert len(self.waveforms) == len(self.isi_dists)
        self.normalize = normalize
        
    def __getitem__(self, idx):
        waveform = torch.as_tensor(self.waveforms[idx, ...]).float()
        isi_dist = torch.as_tensor(self.isi_dists[idx, ...]).float()
        
        if self.normalize:
            waveform = (waveform - waveform.mean()) / waveform.std()
#             isi_dist = (isi_dist - isi_dist.mean()) / isi_dist.std()

        return waveform.unsqueeze(0), isi_dist.unsqueeze(0)
    
    def __len__(self):
        return len(self.waveforms)

In [6]:
class ContrastiveLoss(nn.Module):
    def __init__(self, batch_size, temperature=0.5):
        super().__init__()
        self.batch_size = batch_size
        self.register_buffer("temperature", torch.tensor(temperature))
        self.register_buffer("negatives_mask", (~torch.eye(batch_size * 2, batch_size * 2, dtype=bool)).float())
            
    def forward(self, emb_i, emb_j):
        """
        emb_i and emb_j are batches of embeddings, where corresponding indices are pairs
        z_i, z_j as per SimCLR paper
        """
        z_i = F.normalize(emb_i, dim=1)
        z_j = F.normalize(emb_j, dim=1)

        representations = torch.cat([z_i, z_j], dim=0)
        similarity_matrix = F.cosine_similarity(representations.unsqueeze(1), representations.unsqueeze(0), dim=2)
        
        sim_ij = torch.diag(similarity_matrix, self.batch_size)
        sim_ji = torch.diag(similarity_matrix, -self.batch_size)
        positives = torch.cat([sim_ij, sim_ji], dim=0)

        nominator = torch.exp(positives / self.temperature)
        denominator = self.negatives_mask * torch.exp(similarity_matrix / self.temperature)

        loss_partial = -torch.log(nominator / torch.sum(denominator, dim=1))
        loss = torch.sum(loss_partial) / (2 * self.batch_size)
        return loss


In [7]:
class MixedModel(nn.Module):
    def __init__(self, encoder=None, decoder=None):
        super().__init__()
        
        if encoder is not None:
            self.encoder = encoder
        else:
            self.encoder = nn.Sequential(
                nn.Conv1d(1, 16, 3),
                nn.LeakyReLU(0.01),
                nn.MaxPool1d(3),

                nn.Conv1d(16, 16, 3),
                nn.LayerNorm1d(16),

                nn.LeakyReLU(0.01),
                nn.Conv1d(16, 16, 3),
                nn.MaxPool1d(3),
            )
            self.attn_enc = nn.TransformerEncoder(
                nn.TransformerEncoderLayer(d_model=4, nhead=4, batch_first=True),
                num_layers=4,
                norm=nn.BatchNorm1d(16),
            )
        
        self.fc_wave = nn.Linear(16*(4), 16)
        self.fc_time = nn.Linear(16*(4), 16)
        
        self.norm1 = nn.BatchNorm1d(16)
        
        self.wave_upsample = nn.Linear(16, 16*(4))
        self.time_upsample = nn.Linear(16, 16*(4))
        
        self.norm2 = nn.BatchNorm1d(64)
        
        self.attn_dec = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=(4), nhead=4, batch_first=True),
            num_layers=4,
            norm=nn.BatchNorm1d(16),
        )
        
        if decoder is not None:
            self.decoder = decoder
        else:
            self.decoder = nn.Sequential(
                nn.ConvTranspose1d(16, 16, 3, 2),
                nn.LeakyReLU(0.01),

                nn.ConvTranspose1d(16, 16, 3, 2),
                nn.LeakyReLU(0.01),

                nn.BatchNorm1d(16),
                nn.ConvTranspose1d(16, 1, 3, 3),
            )
        
    def forward(self, wave, time):
        # encoder zone 
        rep_w, rep_t = self.encoder(wave), self.encoder(time)
        rep_w, rep_t = self.attn_enc(rep_w), self.attn_enc(rep_t)
        B, C, H = rep_w.shape
        
        # middle zone 
        rep_w, rep_t = rep_w.view(B, -1), rep_t.view(B, -1)
        rep_w, rep_t = self.fc_wave(rep_w), self.fc_time(rep_t)
        rep_w, rep_t = self.norm1(rep_w), self.norm1(rep_t)
        
        # decoder zone 
        decode_w, decode_t = self.wave_upsample(rep_w), self.time_upsample(rep_t)
        decode_w, decode_t = self.norm2(decode_w), self.norm2(decode_t)
        decode_w, decode_t = decode_w.view(B, C, -1), decode_t.view(B, C, -1)
        decode_w, decode_t = self.attn_dec(decode_w), self.attn_dec(decode_t)
        decode_w, decode_t = self.decoder(decode_w), self.decoder(decode_t)
        decode_w, decode_t = F.interpolate(decode_w, (wave.size(-1),)), F.interpolate(decode_t, (time.size(-1),))

        return rep_w, rep_t, decode_w, decode_t 

In [8]:
model = MixedModel()

sample = torch.randn(2, 8, 1, 50).unbind(0)

model(*sample)

(tensor([[-6.5508e-01,  4.7551e-01,  1.0401e+00,  4.6550e-01,  3.5496e-01,
          -1.6854e+00, -9.1997e-01, -1.1762e+00, -9.1621e-01, -1.0655e+00,
           3.1739e-02,  6.5123e-01,  8.7073e-01, -3.6935e-01,  7.6876e-01,
           5.1133e-01],
         [ 1.1736e+00,  5.5985e-01, -7.0744e-02,  2.0595e+00,  1.0425e+00,
          -1.7530e-01,  1.2845e+00,  2.8852e-01,  1.0757e+00,  1.6372e-01,
           7.8876e-01, -7.8284e-01, -2.4100e-01, -3.4059e-01, -1.1335e-01,
           9.9594e-01],
         [ 1.6199e+00,  9.3750e-01, -1.3342e+00, -5.8227e-01, -2.1810e-03,
           1.5982e-01,  1.8068e-01,  1.7997e+00,  1.4308e+00, -3.9621e-01,
           8.7489e-01, -5.2315e-01, -1.3827e+00,  1.5727e+00, -1.4794e+00,
          -1.3901e+00],
         [ 4.5303e-01, -2.3526e+00, -1.7229e+00, -1.5439e+00, -1.0900e-02,
           2.1223e+00,  1.7842e+00,  1.0295e+00,  9.1395e-01,  2.3417e+00,
           1.6700e+00, -2.1403e+00,  4.6682e-01,  1.5459e-01, -1.5381e+00,
           5.2584e-01],
    

In [12]:
from pytorch_lightning.utilities import grad_norm

class MultimodalEmbedding(pl.LightningModule):
    def __init__(self, base_model, batch_size, contrastive_loss_weight=1.0):
        super().__init__()
        self.model = base_model
        self.contrastive_loss_weight = contrastive_loss_weight
        self.mse_loss = nn.MSELoss()
        self.contrastive_loss = ContrastiveLoss(batch_size, temperature=0.5)

    def training_step(self, batch, batch_idx):
        wave, time = batch
        rep_w, rep_t, decode_w, decode_t = self.model(wave, time)

        mse_loss_w = self.mse_loss(decode_w, wave)
        mse_loss_t = self.mse_loss(decode_t, time)
        mse_loss = mse_loss_w + mse_loss_t

        emb_i, emb_j = rep_w, rep_t
        contrastive_loss = self.contrastive_loss(emb_i, emb_j)

        # Combine losses
        total_loss = mse_loss + self.contrastive_loss_weight * contrastive_loss

        self.log('train_loss', total_loss)
        self.log('train_mse_loss', mse_loss)
        self.log('train_xe_loss', contrastive_loss)
        
        norms = grad_norm(self.model.encoder, norm_type=2)
        self.log_dict(norms)
        norms = grad_norm(self.model.decoder, norm_type=2)
        self.log_dict(norms)
        norms = grad_norm(self.model.fc_wave, norm_type=2)
        self.log_dict(norms)

        return total_loss

    def validation_step(self, batch, batch_idx):
        wave, time = batch
        rep_w, rep_t, decode_w, decode_t = self.model(wave, time)

        mse_loss_w = self.mse_loss(decode_w, wave)
        mse_loss_t = self.mse_loss(decode_t, time)
        mse_loss = mse_loss_w + mse_loss_t

        emb_i, emb_j = rep_w, rep_t
        contrastive_loss = self.contrastive_loss(emb_i, emb_j)

        # Combine losses
        total_loss = mse_loss + self.contrastive_loss_weight * contrastive_loss

        self.log('val_loss', total_loss)
        self.log('val_mse_loss', mse_loss)
        self.log('val_xe_loss', contrastive_loss)

        return total_loss
    
    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=0.001)
        return optimizer

In [13]:
import wandb
wandb.init(reinit=True)
bs = 64t
wf_train, isi_train = waveforms[:15000], isi_dist[:15000]
wf_val, isi_val = waveforms[15000:], isi_dist[15000:]

traindata = EphysDataset(wf_train, isi_train)
valdata = EphysDataset(wf_val, isi_val)

train_loader = DataLoader(traindata, batch_size=bs, drop_last=True)
val_loader = DataLoader(valdata, batch_size=bs, drop_last=True)

base_model = MixedModel()
model = MultimodalEmbedding(base_model=base_model, batch_size=bs, contrastive_loss_weight=0.5)

trainer = pl.Trainer(
    logger=pl.loggers.WandbLogger(),
#     limit_train_batches=1,
    log_every_n_steps=5,
)

trainer.fit(model, train_loader, val_loader)

epoch,▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆███
grad_2.0_norm/0.bias,█▃▃▃▃▂▂▂▁▂▁▁▂▁▁▂▁▂▂▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▂
grad_2.0_norm/0.weight,█▃▅▄▅▄▃▃▃▃▁▂▆▂▂▃▃▄▂▁▁▄▃▁▁▂▄▂▁▂▃▂▁▁▁▃▃▁▂▅
grad_2.0_norm/2.bias,█▂▃▁▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
grad_2.0_norm/2.weight,█▃▃▄▄▂▁▂▃▂▁▂▃▂▂▂▁▃▂▁▁▂▂▁▁▁▅▂▁▂▂▁▁▁▁▁▃▂▂▄
grad_2.0_norm/3.bias,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
grad_2.0_norm/3.weight,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
grad_2.0_norm/4.bias,█▄▅▄▃▄▁▁▄▅▂▂▂▃▂▃▁▂▃▁▁▄▂▁▂▂▇▄▁▃▄▃▃▂▂▂█▃▂▃
grad_2.0_norm/4.weight,█▃▂▃▂▂▁▁▁▂▁▁▃▂▂▁▁▃▂▁▁▂▂▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▂
grad_2.0_norm/5.bias,▃▂▂▃▂▄▁▁▅▅▂▂▁▄▃▄▁▂▃▁▂▅▂▁▂▂█▂▁▃▇▄▃▂▂▂▇▃▃▄
grad_2.0_norm/5.weight,█▂▂▃▂▂▁▁▄▄▁▁▄▂▃▁▂▄▂▁▂▄▃▁▁▁▅▃▁▂▃▃▂▁▁▂▄▂▂▂


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/julian/miniconda3/envs/braingeneers/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
/Users/julian/miniconda3/envs/braingeneers/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

  | Name             | Type            | Params
-----------------------------------------------------
0 | model            | MixedModel      | 156 K 
1 | mse_loss         | MSELoss         | 0     
2 | contrastive_loss | ContrastiveLoss | 0     
-----------------------------------------------------
156 K     Trainable params
0   

Sanity Checking DataLoader 0:   0%|                                                                                             | 0/2 [00:00<?, ?it/s]

/Users/julian/miniconda3/envs/braingeneers/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


/Users/julian/miniconda3/envs/braingeneers/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


...

Epoch 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 234/234 [01:37<00:00,  2.41it/s, v_num=r0gc]
Validation: |                                                                                                                   | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   5%|████▎                                                                                   | 7/142 [00:00<00:17,  7.86it/s]


Validation DataLoader 0:  42%|████████████████████████████████████▊                                                  | 60/142 [00:07<00:10,  8.08it/s]


Validation DataLoader 0:  80%|████████████████████████████████████████████████████████████████████▍                 | 113/142 [00:13<00:03,  8.18it/s]


Epoch 10:  43%|██████████████████████████████████████▍                                                  | 101/234 [00:45<01:00,  2.20it/s, v_num=r0gc]

/Users/julian/miniconda3/envs/braingeneers/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
